<a href="https://colab.research.google.com/github/bairagidotcom/Code-Base/blob/main/RNN_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM


In [ ]:
# ! pip install kaggle
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle competitions download  plant-pathology-2021-fgvc8
!unzip plant*

Streaming output truncated to the last 5000 lines.
  inflating: train_images/9a0b2a6175d666ce.jpg  
  inflating: train_images/9a0ece7b81b8b32a.jpg  
  inflating: train_images/9a1462ce99fa117e.jpg  
  inflating: train_images/9a18cd59c633c0bf.jpg  
  inflating: train_images/9a223715553a6eae.jpg  
  inflating: train_images/9a2b3bf42d4e98d0.jpg  
  inflating: train_images/9a2e269c1969bfe0.jpg  
  inflating: train_images/9a2f3cd04f7fa082.jpg  
  inflating: train_images/9a35774c2aa6207f.jpg  
  inflating: train_images/9a35e26ce68879aa.jpg  
  inflating: train_images/9a4b9bf1544567c4.jpg  
  inflating: train_images/9a55474ae85f0f07.jpg  
  inflating: train_images/9a5936a247bc49e9.jpg  
  inflating: train_images/9a5f6761d68d8a90.jpg  
  inflating: train_images/9a5f6acd4d2045b6.jpg  
  inflating: train_images/9a618622d5bb7599.jpg  
  inflating: train_images/9a6328ee7ae70c1c.jpg  
  inflating: train_images/9a636e1c3c0496ef.jpg  
  inflating: train_images/9a6435c665b4c23f.jpg  
  inflating: train

In [ ]:
# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

In [ ]:
# Embedding dimensions.
row_hidden = 128
col_hidden = 128

In [ ]:

# The data, split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# Reshapes data to 4D for Hierarchical RNN.

# HRNNs can learn across multiple levels of temporal hierarchy over a complex sequence. Usually, the first recurrent layer of an HRNN encodes a sentence
# (e.g. of word vectors) into a sentence vector. The second recurrent layer then encodes a sequence of such vectors (encoded by the first layer) into a 
# document vector. This document vector is considered to preserve both the word-level and sentence-level structure of the context.
 
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))




In [ ]:
# Encodes a row of pixels using TimeDistributed Wrapper.
# TimeDistributed. This wrapper applies a layer to every temporal slice of an input. The input should be at least 3D, and the dimension of index one will be 
# considered to be the temporal dimension. Consider a batch of 32 samples, where each sample is a sequence of 10 vectors of 16 dimensions

encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)




In [ ]:
# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))



In [ ]:
# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])